# Introduktion till sannolikhet och statistik
I denna anteckningsbok kommer vi att leka med några av de koncept som vi tidigare har diskuterat. Många koncept från sannolikhet och statistik är väl representerade i stora bibliotek för databehandling i Python, såsom `numpy` och `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Slumpmässiga variabler och fördelningar
Låt oss börja med att dra ett stickprov på 30 värden från en uniform fördelning från 0 till 9. Vi kommer också att beräkna medelvärde och varians.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

För att visuellt uppskatta hur många olika värden som finns i urvalet kan vi rita ett **histogram**:


In [ ]:
plt.hist(sample)
plt.show()

## Analysera verkliga data

Medelvärde och varians är mycket viktiga när man analyserar verkliga data. Låt oss ladda data om basebollspelare från [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Vi använder ett paket som heter [**Pandas**](https://pandas.pydata.org/) här för dataanalys. Vi kommer att prata mer om Pandas och att arbeta med data i Python senare i den här kursen.

Låt oss beräkna genomsnittsvärden för ålder, längd och vikt:


In [ ]:
df[['Age','Height','Weight']].mean()

Låt oss nu fokusera på höjd och beräkna standardavvikelse och varians:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Förutom medelvärdet är det meningsfullt att titta på medianvärdet och kvartilerna. De kan visualiseras med ett **låddiagram**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Vi kan också göra låddiagram av delmängder av vår dataset, till exempel grupperade efter spelarroll.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Notera**: Detta diagram antyder att första basmän generellt är längre än andra basmän. Senare kommer vi att lära oss hur vi kan testa denna hypotes mer formellt, och hur vi kan visa att våra data är statistiskt signifikanta för att påvisa detta.  

Ålder, längd och vikt är alla kontinuerliga stokastiska variabler. Vad tror du att deras fördelning är? Ett bra sätt att ta reda på det är att plotta histogrammet av värdena: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normalfördelning

Låt oss skapa ett artificiellt urval av vikter som följer en normalfördelning med samma medelvärde och varians som våra verkliga data:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Eftersom de flesta värden i verkliga livet är normalt fördelade, bör vi inte använda en uniform slumptalsgenerator för att generera provdata. Här är vad som händer om vi försöker generera vikter med en uniform fördelning (genererad av `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Konfidensintervall

Låt oss nu beräkna konfidensintervall för vikterna och längderna hos basebollspelare. Vi kommer att använda koden [från denna stackoverflow-diskussion](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Hypotesprövning

Låt oss utforska olika roller i vår basebollspelar-dataset:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Låt oss testa hypotesen att försteklassmän är längre än andraklassmän. Det enklaste sättet att göra detta är att testa konfidensintervallen:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Vi kan se att intervallen inte överlappar.

Ett statistiskt mer korrekt sätt att bevisa hypotesen är att använda ett **Student t-test**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

De två värdena som returneras av funktionen `ttest_ind` är:
* p-värdet kan betraktas som sannolikheten för att två fördelningar har samma medelvärde. I vårt fall är det mycket lågt, vilket betyder att det finns starka bevis som stöder att förstebasmän är längre.
* t-värdet är det mellanliggande värdet av normaliserad medelskillnad som används i t-testet, och det jämförs med ett tröskelvärde för ett givet konfidensvärde.


## Simulera en normalfördelning med centrala gränsvärdessatsen

Den pseudotillfälliga generatorn i Python är utformad för att ge oss en jämn fördelning. Om vi vill skapa en generator för normalfördelning kan vi använda centrala gränsvärdessatsen. För att få ett normalt fördelat värde beräknar vi helt enkelt medelvärdet av ett uniformt genererat urval.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korrelations- och Evil Baseball Corp

Korrelations möjliggör för oss att hitta relationer mellan datasekvenser. I vårt leksaksexempel, låtsas att det finns ett ondskefullt basebollföretag som betalar sina spelare utifrån deras längd - ju längre spelaren är, desto mer pengar får han/hon. Antag att det finns en grundlön på 1000 dollar, och en extra bonus från 0 till 100 dollar, beroende på längd. Vi tar riktiga spelare från MLB och beräknar deras imaginära löner:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Låt oss nu beräkna kovarians och korrelation för dessa sekvenser. `np.cov` ger oss en så kallad **kovariansmatris**, som är en utvidgning av kovarians till flera variabler. Elementet $M_{ij}$ i kovariansmatrisen $M$ är en korrelation mellan indata variablerna $X_i$ och $X_j$, och diagonala värden $M_{ii}$ är variansen för $X_{i}$. På samma sätt ger `np.corrcoef` oss **korrelationsmatrisen**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

En korrelation som är lika med 1 betyder att det finns en stark **linjär relation** mellan två variabler. Vi kan visuellt se den linjära relationen genom att plotta ett värde mot det andra:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Låt oss se vad som händer om relationen inte är linjär. Anta att vårt företag beslutade att dölja det uppenbara linjära sambandet mellan längder och löner, och införde någon icke-linjär funktion i formeln, som till exempel `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

I det här fallet är korrelationen något mindre, men den är fortfarande ganska hög. Nu, för att göra sambandet ännu mindre uppenbart, kanske vi vill lägga till lite extra slumpmässighet genom att lägga till en slumpvariabel till lönen. Låt oss se vad som händer:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Kan du gissa varför prickarna bildar vertikala linjer på detta sätt?

Vi har observerat sambandet mellan ett konstgjort framtaget koncept som lön och den observerade variabeln *längd*. Låt oss även se om de två observerade variablerna, såsom längd och vikt, korrelerar också:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Tyvärr fick vi inga resultat – endast några märkliga `nan`-värden. Detta beror på att några av värdena i vår serie är odefinierade, representerade som `nan`, vilket gör att resultatet av operationen också blir odefinierat. Genom att titta på matrisen kan vi se att `Weight` är den problematiska kolumnen, eftersom självkorrelationen mellan `Height`-värden har beräknats.

> Detta exempel visar vikten av **datapreparering** och **rengöring**. Utan korrekt data kan vi inte beräkna något.

Låt oss använda metoden `fillna` för att fylla i de saknade värdena och beräkna korrelationen:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Det finns faktiskt en korrelation, men inte en så stark som i vårt artificiella exempel. Om vi tittar på spridningsdiagrammet för ett värde mot det andra, skulle relationen vara mycket mindre uppenbar:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Slutsats

I denna anteckningsbok har vi lärt oss hur man utför grundläggande operationer på data för att beräkna statistiska funktioner. Vi vet nu hur man använder ett gediget verktyg av matematik och statistik för att bevisa några hypoteser, och hur man beräknar konfidensintervall för godtyckliga variabler baserat på ett dataurval.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfriskrivning**:
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för några missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
